In [40]:
import pandas as pd
import numpy as np
from collections import Counter
from preprocess_retail import first_change, create_features_for_demand, create_features_by_customer
from tqdm import tqdm_notebook as tqdm
from HyperoptOptimisation import HPOpt
import seaborn as sns
pd.set_option('display.max_columns', 500)

In [5]:
retail = pd.read_excel(r'D:\ML_Sets\online_retail_II.xlsx')

In [6]:
retail, features_for_demand = first_change(retail)

In [45]:
customer_data = create_features_by_customer(retail)

In [8]:
customer_data.head()

,10_hours,11_hours,12 PENCILS SMALL TUBE RED SPOTTY,12 PENCILS SMALL TUBE SKULL,12_hours,13_hours,14_hours,15_hours,16_hours,17_hours,18_hours,19_hours,20_hours,21_hours,3 HEARTS HANGING DECORATION RUSTIC,3 STRIPEY MICE FELTCRAFT,4 TRADITIONAL SPINNING TOPS,6 RIBBONS RUSTIC CHARM,60 CAKE CASES VINTAGE CHRISTMAS,60 TEATIME FAIRY CAKE CASES,72 SWEETHEART FAIRY CAKE CASES,7_hours,8_hours,9_hours,ALARM CLOCK BAKELIKE RED,ANTIQUE LILY FAIRY LIGHTS,ANTIQUE SILVER TEA GLASS ENGRAVED,ANTIQUE SILVER TEA GLASS ETCHED,AREA PATROLLED METAL SIGN,ASSORTED COLOUR BIRD ORNAMENT,ASSORTED COLOUR MINI CASES,ASSORTED COLOURS SILK FAN,ASSTD DESIGN BUBBLE GUM RING,BAKING SET 9 PIECE RETROSPOT,BAKING SET SPACEBOY DESIGN,BATHROOM METAL SIGN,BEWARE OF THE CAT METAL SIGN,BINGO SET,BLACK HEART CARD HOLDER,BLUE 3 PIECE MINI DOTS CUTLERY SET,BLUE HARMONICA IN BOX,BOX OF 24 COCKTAIL PARASOLS,CAKE STAND 3 TIER MAGIC GARDEN,CERAMIC CAKE DESIGN SPOTTED MUG,CERAMIC HEART FAIRY CAKE MONEY BANK,CERAMIC STRAWBERRY CAKE MONEY BANK,CHARLOTTE BAG SUKI DESIGN,CHICK GREY HOT WATER BOTTLE,CHILLI LIGHTS,CHOCOLATE HOT WATER BOTTLE,CHOCOLATE THIS WAY METAL SIGN,CLOTHES PEGS RETROSPOT PACK 24,COFFEE MUG CAT + BIRD DESIGN,COLOUR GLASS T-LIGHT HOLDER HANGING,COLOUR GLASS. STAR T-LIGHT HOLDER,COOK WITH WINE METAL SIGN,CREAM CUPID HEARTS COAT HANGER,CREAM HEART CARD HOLDER,CREAM SWEETHEART MINI CHEST,Country,Customer ID,DOLLY GIRL LUNCH BOX,DOOR MAT FAIRY CAKE,DOOR MAT NEW ENGLAND,DOOR MAT RED SPOT,DOOR MAT UNION FLAG,DOORMAT HEARTS,DOORMAT UNION FLAG,DOORSTOP RETROSPOT HEART,EDWARDIAN PARASOL BLACK,EDWARDIAN PARASOL NATURAL,EDWARDIAN PARASOL RED,ENGLISH ROSE DESIGN PEG BAG,FAWN BLUE HOT WATER BOTTLE,FELTCRAFT 6 FLOWER FRIENDS,FELTCRAFT BUTTERFLY HEARTS,FELTCRAFT CUSHION BUTTERFLY,FELTCRAFT CUSHION OWL,FELTCRAFT CUSHION RABBIT,FELTCRAFT DOLL EMILY,FELTCRAFT DOLL MOLLY,FELTCRAFT DOLL ROSIE,FILIGRIS HEART WITH BUTTERFLY,FIRST AID TIN,FOOD CONTAINER SET 3 LOVE HEART,FOUR HOOK WHITE LOVEBIRDS,Fine price,GARDENERS KNEELING PAD,GIN + TONIC DIET METAL SIGN,GINGHAM HEART DOORSTOP RED,GLASS HONEYPOT WASP CATCHER,GLITTER BUTTERFLY CLIPS,GRAND CHOCOLATECANDLE,GREY HEART HOT WATER BOTTLE,GUMBALL COAT RACK,HAND OVER THE CHOCOLATE SIGN,HAND WARMER BIRD DESIGN,HAND WARMER OWL DESIGN,HAND WARMER RED RETROSPOT,HAND WARMER SCOTTY DOG DESIGN,HAND WARMER UNION JACK,HANGING HEART JAR T-LIGHT HOLDER,HANGING HEART ZINC T-LIGHT HOLDER,HANGING JAM JAR T-LIGHT HOLDER,HANGING METAL HEART LANTERN,HEART FILIGREE DOVE SMALL,HEART FILIGREE DOVE LARGE,HEART IVORY TRELLIS LARGE,HEART IVORY TRELLIS SMALL,HEART OF WICKER LARGE,HEART OF WICKER SMALL,HOME BUILDING BLOCK WORD,HOME SWEET HOME METAL SIGN,HOME SWEET HOME MUG,HOT BATHS METAL SIGN,HOT WATER BOTTLE I AM SO POORLY,HOT WATER BOTTLE TEA AND SYMPATHY,High price,I CAN ONLY PLEASE ONE PERSON MUG,I'M ON HOLIDAY METAL SIGN,IVORY HANGING DECORATION HEART,IVORY KITCHEN SCALES,JUMBO BAG BAROQUE BLACK WHITE,JUMBO BAG OWLS,JUMBO BAG PINK VINTAGE PAISLEY,JUMBO BAG PINK WITH WHITE SPOTS,JUMBO BAG RED RETROSPOT,JUMBO BAG RED WHITE SPOTTY,JUMBO BAG SCANDINAVIAN PAISLEY,JUMBO BAG SPACEBOY DESIGN,JUMBO BAG STRAWBERRY,JUMBO BAG TOYS,JUMBO BAG WOODLAND ANIMALS,JUMBO SHOPPER VINTAGE RED PAISLEY,JUMBO STORAGE BAG SKULLS,JUMBO STORAGE BAG SUKI,"KEY FOB , BACK DOOR","KEY FOB , GARAGE DESIGN","KEY FOB , SHED",KITCHEN METAL SIGN,KNITTED UNION FLAG HOT WATER BOTTLE,LADIES & GENTLEMEN METAL SIGN,LARGE CAKE STAND HANGING STRAWBERY,LARGE HEART MEASURING SPOONS,LARGE POPCORN HOLDER,LAVENDER SCENTED FABRIC HEART,LOVE BUILDING BLOCK WORD,LOVEBIRD HANGING DECORATION WHITE,LUNCH BAG BLACK SKULL.,LUNCH BAG CARS BLUE,LUNCH BAG PINK RETROSPOT,LUNCH BAG RED SPOTTY,LUNCH BAG SPACEBOY DESIGN,LUNCH BAG SUKI DESIGN,LUNCH BAG WOODLAND,LUNCH BOX WITH CUTLERY RETROSPOT,Low price,Lowest price,MEMO BOARD COTTAGE DESIGN,MEMO BOARD RETROSPOT DESIGN,METAL 4 HOOK HANGER FRENCH CHATEAU,METAL SIGN TAKE IT OR LEAVE IT,MINI CAKE STAND HANGING STRAWBERY,NATURAL SLATE HEART CHALKBOARD,NATURAL SLATE RECTANGLE CH

In [10]:
#Создадим словари для категорийных признаков
dict_last_buy = dict(set(zip(customer_data['last_buy'].unique(), np.arange(len(customer_data['last_buy'].unique())))))
dict_country = dict(set(zip(customer_data['Country'].unique(), np.arange(len(customer_data['Country'].unique())))))

In [11]:
customer_data['last_buy'] = customer_data['last_buy'].map(dict_last_buy)
customer_data['Country'] = customer_data['Country'].map(dict_country)

In [15]:
def cancel_or_not(sum_sales):
    if sum_sales>0:
        cancel=0
    else:
        cancel=1
        
    return cancel
    

In [16]:
customer_data['Cancel'] = customer_data['sum_sales'].apply(cancel_or_not)

Последняя покупка все равно показывает перекос даже если брать первые 300 популярных товаров. Тут нужно или брать в признаки все уникальные товары (4000+) или группировать по категориям товары. 

Отмену транзакции предсказывать бесмысленно, т.к. данные уже включают ее. Такой признак надо предсказывать до предстоящей покупки, исходя из поведения клиента на сайте и его местоположения.



In [38]:
customer_data['Cancel'].value_counts()

0    4270
1      48
Name: Cancel, dtype: int64

In [39]:
customer_data['last_buy'].value_counts()[:5]

0     2643
28     101
82      43
14      35
35      33
Name: last_buy, dtype: int64

In [37]:
customer_data['avg_sales'].nunique()

284